#Taller 1:

In [88]:
import nltk
import requests
import pandas as pd
from bs4 import BeautifulSoup
from textblob import TextBlob
from googletrans import Translator
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# URL https://www.mercadolibre.com.co/ofertas

In [89]:
Productos_buscados = 100
url = 'https://www.mercadolibre.com.co/ofertas'

In [90]:

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    enlaces = soup.find_all("a")
    
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")
    

In [91]:
links = []
nombre = []
precio = []
vendedor = []
caracteristicas = []
calificacion_v = []
comentarios = []
estado = []
c_ventas = []
calificacion_p = []
c_opiniones = []


Productos_buscados = 100
request = requests.get(url)
soup = BeautifulSoup(request.text, "html.parser")


for product in soup.find_all('div', {'class': 'items-with-smart-groups'}):
  for link in product.find_all('a', {'class': 'poly-component__title'}):
    while len(links)<= Productos_buscados:
      links.append(link.get('href'))
      
      break
    
    
for entry in links:
  request = requests.get(entry)
  soup = BeautifulSoup(request.text, "html.parser")
  for product in soup.find_all('h1', {'class':'ui-pdp-title'}):
    if product:
        nombre.append(product.text.strip())
    else:
        print(f"[AVISO] No se encontró el nombre del producto en: {entry}")
    
    
  for valor in soup.find_all('div', {'class':'ui-pdp-price__second-line'}):
    for price in valor.find_all('span', {'class':'andes-money-amount__fraction'}):
      if price:
            precios = int(price.text.strip().replace('.', ''))
            precio.append(precios)
      else:
          print(f"[AVISO] No se encontró el precio en: {entry}")
      
      
  for seller in soup.find_all('span', {'class':'ui-pdp-seller__label-text-with-icon'}):
    vendedor.append(seller.text)
  for seller_2 in soup.find_all('div', {'class':'ui-pdp-seller__header__title'}):
    for seller_3 in seller_2.find_all('span', {'class':""}):
      if seller_3:
                vendedor.append(seller_3.text.strip())
      else:
                print(f"[AVISO] No se encontró el vendedor en: {entry}")

  
  for description in soup.find_all('p', {'class':'ui-pdp-description__content'}):
    if description:
        caracteristicas.append(description.text.strip())
    else:
        print(f"[AVISO] No se encontró la descripción en: {entry}")

  
  for rating in soup.find_all('ul', {'class':'ui-seller-data-status__thermometer thermometer-large'}):
    if rating:
        calificacion_v.append(rating.get('value'))
    else:
        print(f"[AVISO] No se encontró la calificación en: {entry}")
    
    
  for comments in soup.find_all('div', {'class':'ui-review-capability-comments'}):
    particion = []
    if comments:
      for comments_2 in comments.find_all('div'):
        for comments_3 in comments_2.find_all('p', {'class':'ui-review-capability-comments__comment__content ui-review-capability-comments__comment__content'}):
          particion.append(comments_3.text.strip())
      comentarios.append(' '.join(particion))
    else:
        print(f"[AVISO] No se encontró la sección de comentarios en: {entry}")
  
  
  for sells in soup.find_all('span', {'class': 'ui-pdp-subtitle'}):
    texto = sells.text.strip()
    if "|" in texto:
        partes = texto.split('|')
        estado_v = partes[0].strip()
        ventas = partes[1].strip()
        ventas = ventas.replace('+', '').replace('vendidos', '').strip()
        if "mil" in ventas:
            ventas = ventas.replace("mil", "").strip()
            if ventas == "":
                ventas = 1000
            else:
                ventas = int(ventas) * 1000
        else:
            try:
                ventas = int(ventas)
            except ValueError:
                print(f"[AVISO] No se pudo convertir ventas a número: {ventas}")
                ventas = 0
        estado.append(estado_v)
        c_ventas.append(ventas)
    else:
        print(f"[AVISO] No se encontró el delimitador '|' en el texto: {texto}")
    
    
  for valoration in soup.find_all('span', {'class':'ui-pdp-review__rating'}):
    if valoration:
      calificacion_p.append(valoration.text)
    else:
        print(f"[AVISO] No se encontró la sección de comentarios en: {entry}")
  
    
  for opinions in soup.find_all('span', {'class':'ui-pdp-review__amount'}):
    if opinions:
      c_opiniones.append(opinions.text.strip('()'))
    else:
        print(f"[AVISO] No se encontró la sección de comentarios en: {entry}")
        
          
product_df = pd.DataFrame(zip(nombre,precio,c_ventas,calificacion_p,c_opiniones,vendedor,caracteristicas,calificacion_v,comentarios),
                          columns = ['Nombre', 'Precio','Cantidad Ventas','Calificacion Producto','Cantidad Opiniones', 'Vendedor', 'Descripcion', 'Rating Vendedor', 'Comentarios'])
product_df.to_csv("ecommerce.csv",index=False)

In [92]:
product_df

,Nombre,Precio,Cantidad Ventas,Calificacion Producto,Cantidad Opiniones,Vendedor,Descripcion,Rating Vendedor,Comentarios
0,Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB neg...,1304900,1000,4.9,2095,CELUMOVIL STORE,Fotografía profesional en tu bolsilloDescubre ...,5,"Una bestia de celular, corre todos los juegos ..."
1,Nintendo Switch Oled 64gb Standard Color Rojo ...,1189900,1000,4.9,1919,CELUMOVIL STORE,La consola Switch ofrece entretenimiento diari...,5,
2,Tarjeta de video Nvidia MSI Ventus 2X GeForce...,1469900,100,4.9,78,TAURET.COMPUTADORES,Este componente electrónico procesa la informa...,5,Genial rendimiento y eso que no he actualizado...
3,Hot Wheels City Pista De Juguete Ultimate Gara...,577900,100,4.9,28,MATTEL COLOMBIA,Hot Wheels City Pista De Juguete Ultimate Gara...,5,Excelente tal y como lo esperaba. Muy buen pro...
4,Jbl Partybox Encore Essential - Altavoz De Fie...,909900,100,4.8,85,IXCOMERCIO COLOMBIA,JBL PartyBox Encore Essential - Altavoz portát...,5,"Muy buena relación precio calidad y potencia, ..."
5,Audífonos Sony Bluetooth Noise Cancelling | Wh...,1187900,1000,4.8,726,SONY COLOMBIA,Los audífonos WH-1000XM5 brindan una experienc...,5,Nunca tuve la oportunidad de usar unos audífon...
6,Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB ama...,1304900,1000,4.9,2095,CELUMOVIL STORE,Fotografía profesional en tu bolsilloDescubre ...,5,"Una bestia de celular, corre todos los juegos ..."
7,Impresora multifunción a color Epson EcoTank l...,1179900,500,4.7,180,Epson,Epson busca que sus clientes obtengan el máxim...,5,"Hasta el momento todo marcha muy bien, gracias..."
8,Nintendo Switch Oled 64gb Color Blanco/negro,1189900,1000,4.9,1919,CELUMOVIL STORE,Aviso legal• La duración de la batería depende...,5,Esta version del nintendo switch es del reino ...
9,Impresora multifunción a color Brother InkBene...,590000,500,4.7,245,NEW TECNOLOGIES,Brother ofrece productos de alta calidad y con...,5,"Muy buen producto, algunas caracteristicas que..."


In [93]:
df_total = pd.read_csv('ecommerce.csv')

In [94]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Nombre                 53 non-null     object 
 1   Precio                 53 non-null     int64  
 2   Cantidad Ventas        53 non-null     int64  
 3   Calificacion Producto  53 non-null     float64
 4   Cantidad Opiniones     53 non-null     int64  
 5   Vendedor               53 non-null     object 
 6   Descripcion            53 non-null     object 
 7   Rating Vendedor        53 non-null     int64  
 8   Comentarios            51 non-null     object 
dtypes: float64(1), int64(4), object(4)
memory usage: 3.9+ KB


In [95]:
no_null = df_total[df_total["Comentarios"].notna()]
data = no_null.reset_index()
df = data.drop(columns=['index'])

In [96]:
df

,Nombre,Precio,Cantidad Ventas,Calificacion Producto,Cantidad Opiniones,Vendedor,Descripcion,Rating Vendedor,Comentarios
0,Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB neg...,1304900,1000,4.9,2095,CELUMOVIL STORE,Fotografía profesional en tu bolsilloDescubre ...,5,"Una bestia de celular, corre todos los juegos ..."
1,Tarjeta de video Nvidia MSI Ventus 2X GeForce...,1469900,100,4.9,78,TAURET.COMPUTADORES,Este componente electrónico procesa la informa...,5,Genial rendimiento y eso que no he actualizado...
2,Hot Wheels City Pista De Juguete Ultimate Gara...,577900,100,4.9,28,MATTEL COLOMBIA,Hot Wheels City Pista De Juguete Ultimate Gara...,5,Excelente tal y como lo esperaba. Muy buen pro...
3,Jbl Partybox Encore Essential - Altavoz De Fie...,909900,100,4.8,85,IXCOMERCIO COLOMBIA,JBL PartyBox Encore Essential - Altavoz portát...,5,"Muy buena relación precio calidad y potencia, ..."
4,Audífonos Sony Bluetooth Noise Cancelling | Wh...,1187900,1000,4.8,726,SONY COLOMBIA,Los audífonos WH-1000XM5 brindan una experienc...,5,Nunca tuve la oportunidad de usar unos audífon...
5,Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB ama...,1304900,1000,4.9,2095,CELUMOVIL STORE,Fotografía profesional en tu bolsilloDescubre ...,5,"Una bestia de celular, corre todos los juegos ..."
6,Impresora multifunción a color Epson EcoTank l...,1179900,500,4.7,180,Epson,Epson busca que sus clientes obtengan el máxim...,5,"Hasta el momento todo marcha muy bien, gracias..."
7,Nintendo Switch Oled 64gb Color Blanco/negro,1189900,1000,4.9,1919,CELUMOVIL STORE,Aviso legal• La duración de la batería depende...,5,Esta version del nintendo switch es del reino ...
8,Impresora multifunción a color Brother InkBene...,590000,500,4.7,245,NEW TECNOLOGIES,Brother ofrece productos de alta calidad y con...,5,"Muy buen producto, algunas caracteristicas que..."
9,"HOT WHEELS City, Remolque de Carreras, Pista d...",668970,500,4.9,121,REDES YSUMINISTROS,REMOLQUE DE CARRERAS. Guarda los Hot Wheels y ...,5,"Supero mis espectativas, es súper grande y bas..."


In [97]:
polaridad_inicial = []
polaridad_final = []
subjetividad_inicial = []
subjetividad_final = []


stemmer = PorterStemmer()
stemmer2 = SnowballStemmer(language='spanish')
lemmatizer = WordNetLemmatizer()
translator = Translator()
frases = df['Comentarios']
for frase in frases:
  
  blob = TextBlob(frase)
  translation = translator.translate(str(blob), src='es', dest='en').text
  
  
  blob_t = TextBlob(translation)
  sentimiento = blob_t.sentiment
  polaridad = sentimiento.polarity
  subjetividad = sentimiento.subjectivity
  polaridad_inicial.append(polaridad)
  subjetividad_inicial.append(subjetividad)

  
  tokens=word_tokenize(translation.lower())
  prueba_2 = [stemmer2.stem(palabra_2) for palabra_2 in tokens]

  
  prueba_3 = [lemmatizer.lemmatize(palabra_3) for palabra_3 in tokens]

  
  stopwords = nltk.corpus.stopwords.words('english')
  newtokens=[word for word in tokens if word not in stopwords]

  
  
  sentence = ' '.join(newtokens)

  
  translation_2 = translator.translate(str(sentence), src='en', dest='es').text

  
  blob_t = TextBlob(sentence)
  sentimiento_t = blob_t.sentiment
  polaridad_t = sentimiento_t.polarity
  subjetividad_t = sentimiento_t.subjectivity
  polaridad_final.append(polaridad_t)
  subjetividad_final.append(subjetividad_t)
sentimiento_df = pd.DataFrame(zip(polaridad_inicial,subjetividad_inicial,polaridad_final,subjetividad_final), 
                              columns=['Polaridad Inicial','Subjetividad Inicial','Polaridad Final','Subjetividad Final'])

In [98]:
df_final = pd.concat([df,sentimiento_df],axis=1)
df_final

,Nombre,Precio,Cantidad Ventas,Calificacion Producto,Cantidad Opiniones,Vendedor,Descripcion,Rating Vendedor,Comentarios,Polaridad Inicial,Subjetividad Inicial,Polaridad Final,Subjetividad Final
0,Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB neg...,1304900,1000,4.9,2095,CELUMOVIL STORE,Fotografía profesional en tu bolsilloDescubre ...,5,"Una bestia de celular, corre todos los juegos ...",0.235714,0.637143,0.260000,0.620000
1,Tarjeta de video Nvidia MSI Ventus 2X GeForce...,1469900,100,4.9,78,TAURET.COMPUTADORES,Este componente electrónico procesa la informa...,5,Genial rendimiento y eso que no he actualizado...,0.401429,0.528571,0.472000,0.578000
2,Hot Wheels City Pista De Juguete Ultimate Gara...,577900,100,4.9,28,MATTEL COLOMBIA,Hot Wheels City Pista De Juguete Ultimate Gara...,5,Excelente tal y como lo esperaba. Muy buen pro...,0.633333,0.633333,0.850000,0.800000
3,Jbl Partybox Encore Essential - Altavoz De Fie...,909900,100,4.8,85,IXCOMERCIO COLOMBIA,JBL PartyBox Encore Essential - Altavoz portát...,5,"Muy buena relación precio calidad y potencia, ...",0.742000,0.676000,0.700000,0.640000
4,Audífonos Sony Bluetooth Noise Cancelling | Wh...,1187900,1000,4.8,726,SONY COLOMBIA,Los audífonos WH-1000XM5 brindan una experienc...,5,Nunca tuve la oportunidad de usar unos audífon...,0.329762,0.521032,0.351515,0.481313
5,Xiaomi POCO Poco X6 Pro 5G Dual SIM 512 GB ama...,1304900,1000,4.9,2095,CELUMOVIL STORE,Fotografía profesional en tu bolsilloDescubre ...,5,"Una bestia de celular, corre todos los juegos ...",0.235714,0.637143,0.260000,0.620000
6,Impresora multifunción a color Epson EcoTank l...,1179900,500,4.7,180,Epson,Epson busca que sus clientes obtengan el máxim...,5,"Hasta el momento todo marcha muy bien, gracias...",0.562500,0.560000,0.714063,0.625000
7,Nintendo Switch Oled 64gb Color Blanco/negro,1189900,1000,4.9,1919,CELUMOVIL STORE,Aviso legal• La duración de la batería depende...,5,Esta version del nintendo switch es del reino ...,0.381360,0.514035,0.355637,0.515686
8,Impresora multifunción a color Brother InkBene...,590000,500,4.7,245,NEW TECNOLOGIES,Brother ofrece productos de alta calidad y con...,5,"Muy buen producto, algunas caracteristicas que...",0.425370,0.586481,0.360370,0.566481
9,"HOT WHEELS City, Remolque de Carreras, Pista d...",668970,500,4.9,121,REDES YSUMINISTROS,REMOLQUE DE CARRERAS. Guarda los Hot Wheels y ...,5,"Supero mis espectativas, es súper grande y bas...",0.644444,0.805556,0.644444,0.805556


$$
Puntaje = 10 \times \left( w1 \times \left( \frac{Cantidad Ventas}{max\_ventas} \right) + w2 \times \left( \frac{Calificacion Producto}{5} \right) + w3 \times \left( \frac{Cantidad Opiniones}{max\_opiniones} \right) + w4 \times \left( \frac{Rating Vendedor}{5} \right) + w5 \times \left( \frac{Polaridad Final + 1}{2} \right) - w6 \times Subjetividad Final \right)
$$



In [99]:
w1, w2, w3, w4, w5, w6 = 0.22, 0.2, 0.15, 0.2, 0.23, 0.1

max_ventas = df_final['Cantidad Ventas'].max()
max_opiniones = df_final['Cantidad Opiniones'].max()

df_final['Puntaje'] = 10 * (
    w1 * (df_final['Cantidad Ventas'] / max_ventas) +
    w2 * (df_final['Calificacion Producto'] / 5) +
    w3 * (df_final['Cantidad Opiniones'] / max_opiniones) +
    w4 * (df_final['Rating Vendedor'] / 5) +
    w5 * ((df_final['Polaridad Final'] + 1) / 2) -
    w6 * df_final['Subjetividad Final']
)

top_10_productos = df_final[['Nombre', 'Puntaje', 'Precio']].sort_values(by='Puntaje', ascending=False).head(10)

pivot_table = top_10_productos.pivot_table(index='Nombre', values=['Puntaje', 'Precio'], aggfunc='mean').sort_values(by='Puntaje', ascending=False)

In [100]:
pivot_table

,Precio,Puntaje
Nombre,,
Disco Solido 500 Gb Crucial Bx500,156400.0,8.312184
Silla Ejecutiva Magnux Ergonómica Negra Con Tapizado Mesh Color Negro,155968.0,8.256004
Hidrolavadora Electrica Trent Hlt203 1400w Alta Presion 1600 Psi 110 Bar Con Accesorios By Femmto,228479.0,7.484651
Audífonos Inalámbricos Wh-ch520 Color Negro,1829900.0,7.344337
Xiaomi Redmi 13C Dual SIM 256 GB midnight black 8 GB RAM,3299000.0,6.890352
Mesa Portafolio Plegable 180cm Resistente Exterior Jardin Color Blanco,528900.0,6.422708
Tenis Cloudfoam Comfy Con Cordones Elásticos Ih2966 adidas,167600.0,6.226555
Nintendo Switch Oled 64gb Color Blanco/negro,1189900.0,6.029373
Portátil Asus Tuf Gaming F15 Intel Core I5 12500h 24gb 512gb Color Gris,149900.0,6.008020
